In [12]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [13]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

Enter a class to grade (i.e. class1 for class1)
File not accessible


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [14]:
def valueCheck(x):
    value = x.split(",")
    length = len(value)
    return length
def has26Values(x):
    if x == 26:
        return True
    else:
        return False
for grade in grades: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                incorrect_id = value[0]
                print("Invalid line of data:", (incorrect_id) + " does not contain exactly 26 values:")
                print(grade)    
values_count = list(map(valueCheck,grades))
values_check = list(map(has26Values,values_count))
print("Total valid lines of data:", values_check.count(True))
print("Total invalid lines of data:", values_check.count(False))


Invalid line of data: N00000616 does not contain exactly 26 values:
N00000616,B,A,D,D,A,B,D,A,C,C,D,C,A,B,A,C,B,D,A,,A,B,B,D,D,A

Invalid line of data: N00000638 does not contain exactly 26 values:
N00000638,B,A,D,D,C,B,D,A,C,,D,B,A,B,A,B,C,D,A,C,A,,,D,D,Q

Total valid lines of data: 493
Total invalid lines of data: 2


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html
https://linuxhint.com/pandas-compare-two-dataframes-row-by-row/
pandas.drop 
https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bool.html 
https://stackoverflow.com/questions/35307732/how-to-sum-in-pandas-by-unique-index-in-several-columns
https://stackoverflow.com/questions/44564414/how-can-i-use-return-to-get-back-multiple-values-from-a-loop-can-i-put-them-i 

--------------------
How to get the incorrect answer to a new dataframe ( make a new dataframe with the quantity of answering incorrectly, score

In [26]:
answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_keys.split(",")
answer_key_table = pd.DataFrame(answer_key)
#answer_key_table = answer_key_table.reset_index(drop=True)
#def checkAnswer(grades, answer_key_table):
for grade in grades:
        value = grade.split(",")
        if len(value) > 25:
                continue
        answer_table = pd.DataFrame(value)
        answer_table = answer_table.iloc[1: , :]
        answer_table = answer_table.reset_index(drop=True)
        cVal = answer_table.iloc[24][0]
        cValSplit = cVal.split("\n")
        c = cValSplit[0]
        answer_table = answer_table.replace([cVal],c)
        answer_table['prices_match'] = np.where(answer_table[0] == answer_key_table[0], 'True', answer_table[0])
  

Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'